### Imports

In [1]:
import requests
import json
import csv
import os
import pandas as pd

### Extract Schools from TennisReporting - Save as JSON file

In [2]:
# --- Configuration ---
main_dir = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
json_path = os.path.join(main_dir, "schools.json")
csv_path = os.path.join(main_dir, "schools.csv")

# --- Step 1: Fetch data from API ---
url = "https://api.v2.tennisreporting.com/schools"
print(f"Fetching school data from {url}...")

response = requests.get(url)
response.raise_for_status()  # Stop if there's an HTTP error
schools_data = response.json()

# --- Step 2: Save JSON to file ---
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(schools_data, f, indent=2, ensure_ascii=False)
print(f"Saved raw JSON data to {json_path}")


Fetching school data from https://api.v2.tennisreporting.com/schools...
Saved raw JSON data to C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.json


### Convert JSON file to a .csv file
### Filter out non-Oregon schools

In [5]:

# --- Step 3: Read JSON and filter for Oregon schools ---
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

filtered_schools = []
for s in data:
    try:
        state_abbr = s.get("city", {}).get("state", {}).get("abbr")
        if state_abbr == "OR":
            school_name = s.get("name", "").strip().upper()
            city_name = s.get("city", {}).get("name", "").strip().upper()
            filtered_schools.append({
                "SchoolID": s.get("id"),
                "School_Name": school_name,
                "City_Name": city_name
            })
    except Exception as e:
        print(f"Skipping record due to error: {e}")

# --- Step 4: Write filtered results to CSV ---
fieldnames = ["SchoolID", "School_Name", "City_Name"]
with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(filtered_schools)

print(f"Wrote {len(filtered_schools)} Oregon schools to {csv_path}")


Skipping record due to error: 'NoneType' object has no attribute 'get'
Skipping record due to error: 'NoneType' object has no attribute 'get'
Skipping record due to error: 'NoneType' object has no attribute 'get'
Wrote 467 Oregon schools to C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.csv


### Print 20 records for visual check

In [8]:
# --- Read CSV ---
df = pd.read_csv(csv_path)

# --- Print first 10 records ---
print("\n=== TOP 10 RECORDS ===")
print(df.head(10).to_string(index=False))

# --- Print last 10 records ---
print("\n=== LAST 10 RECORDS ===")
print(df.tail(10).to_string(index=False))



=== TOP 10 RECORDS ===
 SchoolID                       School_Name  City_Name
    74588            FALLS CITY HIGH SCHOOL FALLS CITY
    74589        OREGON SCHOOL FOR THE DEAF      SALEM
    74590       OREGON SCHOOL FOR THE BLIND      SALEM
    74596                              VALE       VALE
    74600               GERVAIS HIGH SCHOOL    GERVAIS
    74602 DOUGLAS AVENUE ALTERNATIVE SCHOOL    GERVAIS
    74605       YAMHILL CARLTON HIGH SCHOOL    YAMHILL
    74608            HARRISBURG HIGH SCHOOL HARRISBURG
    74614                           STAYTON    STAYTON
    74616    SOUTH WASCO COUNTY HIGH SCHOOL     MAUPIN

=== LAST 10 RECORDS ===
 SchoolID                         School_Name    City_Name
   125029         OPEN DOOR CHRISTIAN ACADEMY    TROUTDALE
   125033           HORIZON CHRISTIAN SCHOOLS     TUALATIN
   125041     MCKENZIE RIVER CHRISTIAN SCHOOL         VIDA
  7070534                 FOUR RIVERS CHARTER      ONTARIO
  7070582                        MOUNTAINSIDE    BE

### Data Quality Checks using check_csv function

In [6]:
from function_scripts.data_quality import check_csv

csv_file = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.csv"

results = check_csv(
    csv_path=csv_file,
    unique_fields=["SchoolID"],         
    null_fields=["School_Name", "SchoolID"],
    whitespace_fields=["School_Name","City_Name"]
)


===== DATA QUALITY CHECKS =====

Reading CSV: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.csv
Loaded 467 rows.

Checking for duplicate values...

✅ No duplicates in 'SchoolID'

---------------------------------------

Checking for null values...

✅ No null values in 'School_Name'
✅ No null values in 'SchoolID'

---------------------------------------

Checking for leading/trailing whitespace...

✅ No leading/trailing whitespace in 'School_Name'
✅ No leading/trailing whitespace in 'City_Name'

===== DATA QUALITY CHECK COMPLETE =====

